# Importing required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('Data to Plot.csv')                           # importing data from data_handling file
data.set_index(data['Timestamp'], inplace = True)
data.drop(['Timestamp', 'Timestamp.1'], axis = 1, inplace=True)  
data.rename(columns = {'Value':'Predicted', 'pm2.5':'Actual'}, inplace = True)  # renaming columns to predicted and actual
data

,Predicted,Actual
Timestamp,,
2018-09-13 16:00:00,1.1836,0.3
2018-09-13 17:00:00,0.8725,2.7
2018-09-13 18:00:00,0.8762,3.2
2018-09-13 19:00:00,1.0171,1.9
2018-09-13 20:00:00,1.4492,3.5
...,...,...
2018-12-31 19:00:00,5.5130,8.8
2018-12-31 20:00:00,5.7150,8.1
2018-12-31 21:00:00,5.9370,8.9


In [15]:
index = data[data.isna().Actual == True].index
data.drop(index,axis=0,inplace=True)
data

,Predicted,Actual
Timestamp,,
2018-09-13 16:00:00,1.1836,0.3
2018-09-13 17:00:00,0.8725,2.7
2018-09-13 18:00:00,0.8762,3.2
2018-09-13 19:00:00,1.0171,1.9
2018-09-13 20:00:00,1.4492,3.5
...,...,...
2018-12-31 19:00:00,5.5130,8.8
2018-12-31 20:00:00,5.7150,8.1
2018-12-31 21:00:00,5.9370,8.9


# Defining model

In [16]:
# setting window_size to the optimum value
window_size = 160  

# Creating Dataset for applying LSTM model

In [17]:
# creating data for the model
x = data['Predicted'].iloc[:window_size]    
x = x.to_numpy()
X = []                      # X contains the first window_size values from Predicted and next value from Actual. Then we concatenate the value to Predicted and take next value from Actual
y = []
for i in range((len(data) - window_size - 1)):
    row = [[a] for a in x[i:i+window_size]]
    X.append(row)
    x = np.concatenate((x, [data['Actual'][i + window_size + 1]]))
    y.append(x[-1])
y = np.array(y)
X = np.array(X)
print(X.shape, y.shape)

(2457, 160, 1) (2457,)


# Separating data into Train, Cross-validation and Test

In [18]:
Train_X = X[:1500]
Train_y = y[:1500]
Val_X = X[1500:2000]
Val_y = y[1500:2000]
Test_X = X[2000:]
Test_y = y[2000:]

# Model

In [19]:
model1 = Sequential()
model1.add(InputLayer((window_size, 1)))
model1.add(LSTM(624))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(1, activation='linear'))

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 624)               1562496   
                                                                 
 dense_2 (Dense)             (None, 16)                10000     
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,572,513
Trainable params: 1,572,513
Non-trainable params: 0
_________________________________________________________________


# Determining minimum error on hyperparameters (Hyperparameter Tuning)

In [20]:
# Using 2 for loops to determine the optimal value of window size and LSTM model units required. Using Errors list to find the errors and the minimum of them will be one with optimal values.

# optimal_window_size = 5 
# optimal_units = 128
# min_error = 1000000
# errors = []
# for window_size in range(150, 180, 5):
#     errors_ =[] 
#     for units_ in range(600, 648, 8):
#         x = data['Predicted'].iloc[:window_size]
#         x = x.to_numpy()
#         X = []
#         y = []
#         for i in range((len(data) - window_size - 1)):
#             row = [[a] for a in x[i:i+window_size]]
#             X.append(row)
#             x = np.concatenate((x, [data['Actual'][i + window_size + 1]]))
#             y.append(x[-1])
#         y = np.array(y)
#         X = np.array(X)
#         print(X.shape, y.shape)

#         Train_X = X[:1500]
#         Train_y = y[:1500]
#         Val_X = X[1500:2000]
#         Val_y = y[1500:2000]
#         Test_X = X[2000:]
#         Test_y = y[2000:]

#         model1 = Sequential()
#         model1.add(InputLayer((window_size, 1)))
#         model1.add(LSTM(units_))
#         model1.add(Dense(16, activation='relu'))
#         model1.add(Dense(1, activation='linear'))

#         model1.summary()

#         model1.compile(loss=MeanSquaredError(), optimizer=Adam(0.01), metrics=[RootMeanSquaredError()])

#         model1.fit(Train_X, Train_y, validation_data = (Val_X, Val_y), epochs = 10)
        
#         train_predictions = model1.predict(X).flatten()
#         train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y})
#         train_results['Train Predictions'] = train_results['Train Predictions'].shift(periods = -1)
#         error = 0
#         for i in range(len(train_results) - 2):
#             error += (train_results['Train Predictions'][i] - train_results['Actuals'][i]) ** 2
#         if error < min_error:
#             optimal_window_size = window_size
#             optimal_units = units_
#             min_error = error
#         errors_.append(error)
#     errors.append(errors_)

In [21]:
model1.compile(loss=MeanSquaredError(), optimizer=Adam(0.001), metrics=[RootMeanSquaredError()])

In [22]:
model1.fit(Train_X, Train_y, validation_data = (Val_X, Val_y), epochs = 10)

Epoch 1/10
47/47 [==============================] - 53s 1s/step - loss: 85.7976 - root_mean_squared_error: 9.2627 - val_loss: 26.7624 - val_root_mean_squared_error: 5.1732
Epoch 2/10
47/47 [==============================] - 52s 1s/step - loss: 31.8980 - root_mean_squared_error: 5.6478 - val_loss: 17.0916 - val_root_mean_squared_error: 4.1342
Epoch 3/10
47/47 [==============================] - 53s 1s/step - loss: 22.2030 - root_mean_squared_error: 4.7120 - val_loss: 17.8756 - val_root_mean_squared_error: 4.2280
Epoch 4/10
47/47 [==============================] - 53s 1s/step - loss: 18.6092 - root_mean_squared_error: 4.3138 - val_loss: 17.4529 - val_root_mean_squared_error: 4.1777
Epoch 5/10
47/47 [==============================] - 53s 1s/step - loss: 14.9038 - root_mean_squared_error: 3.8605 - val_loss: 16.9229 - val_root_mean_squared_error: 4.1137
Epoch 6/10
47/47 [==============================] - 53s 1s/step - loss: 12.8484 - root_mean_squared_error: 3.5845 - val_loss: 17.1264 - val_

In [23]:
train_predictions = model1.predict(X).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y})
train_results.to_csv('train_predictions.csv')

77/77 [==============================] - 31s 398ms/step


# Evaluating error

In [24]:
from math import sqrt
mse_error = 0
avg_error = 0
for i in range(len(train_results) - len(Test_X), len(train_results) - 2):
    mse_error += (train_results['Train Predictions'][i+1] - train_results['Actuals'][i]) ** 2
    avg_error += sqrt((train_results['Train Predictions'][i+1] - train_results['Actuals'][i]) ** 2)
mse_error /= len(Test_X)
avg_error /= len(Test_X)

In [25]:
print(f"Mean Squared error on Test set is {mse_error}")
print(f"Average error on Test set is {avg_error}")

Mean Squared error on Test set is 0.6378786073048666
Average error on Test set is 0.596192701744563
